<hr style="height:2px;">

# Demo: Training data generation for isotropic ...

### Notes

- Documentation available: http://csbdeep.bioimagecomputing.com/doc/

In [ ]:
from __future__ import print_function, unicode_literals, absolute_import, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import os
from tifffile import imread
from csbdeep import datagen
from csbdeep.plot_utils import plot_some

Expect data like this:

    raw_data
    ├── test_stacks_sub_4
    │   └── stack_low_sub_4_03.tif
    └── training_stacks
        ├── high
        │   ├── stack_00.tif
        │   ├── stack_01.tif
        │   └── stack_02.tif
        └── low
            ├── stack_00.tif
            ├── stack_01.tif
            └── stack_02.tif

<hr style="height:2px;">

# Raw 3D image stack with low z resolution

We only show one stack for demonstration, but you should have different stacks at different developmental timepoints.

In [ ]:
y = imread('raw_data/training_stacks/high/stack_00.tif')
x = imread('raw_data/training_stacks/low/stack_00.tif')
axes = 'ZYX'
print('image size =', x.shape)
print('image axes =', axes)

plt.figure(figsize=(15,10))
plot_some(np.stack([x[:,:,0],y[:,:,0]]),title_list=[['low (ZY slice)','high (ZY slice)']], pmin=2,pmax=99.8);

<hr style="height:2px;">

# Dev

In [ ]:
#from csbdeep.plot_utils import to_color
#raw_data = datagen.get_tiff_pairs_from_folders (
#    basepath   = 'raw_data/training_stacks',
#    source_dirs = ['low'],
#    target_dir = 'high',
#    axes = axes,
#)
#iso_transform = datagen.anisotropic_distortions(
#    psf = None,
#    subsample = 4,
#    subsample_axis = 'Z',
#    yield_target = 'target',
#)
#
#gen_in = raw_data.generator()
#gen_in = datagen.crop_images([slice(None),slice(0,256),slice(0,256)]).generator(gen_in)
#gen_iso = iso_transform.generator(gen_in)
#print(iso_transform.name)
#
#for x,y,axes,mask in gen_iso:
#    plt.figure(figsize=(16,6))
#    print(x.shape, y.shape)
#    #a,b = x[10],y[10]
#    a,b = x[:,10],y[:,10]
#    c = to_color(np.stack([a,b]))
#    plt.subplot(131); plt.imshow(a)
#    plt.subplot(132); plt.imshow(b)
#    plt.subplot(133); plt.imshow(c)
#    break
#None;

<hr style="height:2px;">

# Patch generation

From corresponding stacks, we generate 3D patches.

In [ ]:
raw_data = datagen.get_tiff_pairs_from_folders (
    basepath    = 'raw_data/training_stacks',
    source_dirs = ['low'],
    target_dir  = 'high',
    axes        = axes,
)

iso_transform = datagen.anisotropic_distortions(
    psf            = None,
    subsample      = 4,
    subsample_axis = 'Z',
    yield_target   = 'target',
)

X, Y, XY_axes = datagen.create_patches (
    raw_data            = raw_data,
    patch_size          = (64,64,64),
    patch_axes          = 'ZYX',
    n_patches_per_image = 512,
    transforms          = [iso_transform],
    save_file           = 'my_training_data.npz',
)

In [ ]:
assert X.shape == Y.shape
print("shape of X,Y =", X.shape)
print("axes  of X,Y =", XY_axes)

## Show

Show some of those patch pairs (first row: *source*, second row: *target*)

In [ ]:
from csbdeep.plot_utils import plot_some

for i in range(2):
    plt.figure(figsize=(16,4))
    sl = slice(8*i, 8*(i+1))
    sl = slice(8*i, 8*(i+1)), slice(None), slice(None), 0
    plot_some(np.moveaxis(X[sl],1,-1),np.moveaxis(Y[sl],1,-1),title_list=[np.arange(sl[0].start,sl[0].stop)])
    plt.show()
None;